## Introduction

The importance of Carbon Capture, Utilization, and Storage (CCUS) technology is growing as governments take more aggressive steps to manage CO2 in the atmosphere. The feasibility of capturing CO2 particularly depends on the economic viability of the project. 

In theory, the ability to accurately assess future carbon emissions could be a factor in project selection for Carbon Capture. This could have a major impact on point source capture projects, such as working with power plants, factory farms, or landfills. On the other hand, we expect carbon emissions data to have little influence on Direct Air Capture (DAC) projects. This is because *local atmospheric CO2 concentrations vary by little more than 2%* due to the strong influence of atmospheric mixing. 

<p align="center"><img src="./images/co2_surface_conc_map.png" style="width:500px"/></p>

This challenge directs us to predict future carbon emissions in the United States for specific locations, based on prior monthly emissions data for these same locations. The data was sourced from the Copernicus Atmosphere Monitoring Service. The training data includes monthly data for 153 locations from January 2000 through December 2019. We will need to make predictions for the twelve months of 2020.

## Imports and Data Preparation

In [1]:
#Import necessary libraries 
import pandas as pd
import numpy as np
import plotly.express as px
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("data/train.csv")    #get input train dataset
df['month'] = df['month'] + 1  # make month column start at 1 for January instead of 0 (we noticed that the month column start from 0 to 11)
df['date']  = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')
df['loc_uid'] = df['latitude'].astype(str) + ', ' + df['longitude'].astype(str)
df = df[["loc_uid", "latitude", "longitude", "date", "month", "year", "co2"]]
df.head()

,loc_uid,latitude,longitude,date,month,year,co2
0,"35.1, -114.5",35.1,-114.5,2000-01-01,1,2000,24.550266
1,"33.3, -112.8",33.3,-112.8,2000-01-01,1,2000,4.388263
2,"39.5, -112.5",39.5,-112.5,2000-01-01,1,2000,13.314685
3,"36.9, -111.3",36.9,-111.3,2000-01-01,1,2000,16.433954
4,"39.1, -111.1",39.1,-111.1,2000-01-01,1,2000,10.039619


## Monthly Model

Let's see if there are any trends in how monthly CO2 emissions relate to annual emmissions. We calculate for each location the ratio of each months emissions to the annual total emissions for the year.

In [3]:
annual_totals = df.groupby(['loc_uid','year']).sum().reset_index()[["loc_uid", "year", "co2"]]
df = df.merge(annual_totals, left_on=['loc_uid','year'], right_on=['loc_uid','year'], suffixes=('_monthly', '_annual'))
df["ratio"] = df["co2_monthly"] / df["co2_annual"]
df.head()

,loc_uid,latitude,longitude,date,month,year,co2_monthly,co2_annual,ratio
0,"35.1, -114.5",35.1,-114.5,2000-01-01,1,2000,24.550266,267.236373,0.091867
1,"35.1, -114.5",35.1,-114.5,2000-02-01,2,2000,20.931224,267.236373,0.078325
2,"35.1, -114.5",35.1,-114.5,2000-03-01,3,2000,20.514288,267.236373,0.076765
3,"35.1, -114.5",35.1,-114.5,2000-04-01,4,2000,18.495834,267.236373,0.069212
4,"35.1, -114.5",35.1,-114.5,2000-05-01,5,2000,20.739020,267.236373,0.077606


When we plot these ratios of monthly to annual emissions for all of the individual locations, it's clear there are four distinct groupings. Amazingly, these ratios are perfectly consistent. If a location has 10% of it's annual emisions in January one year, it will have 10% of it's annual emissions in January every other year! Moreover, these ratios are shared by multiple locations.

In [4]:
df.sort_values(["loc_uid", "month"], inplace=True)
fig = px.line(df, x="month", y="ratio", color="loc_uid", title="Fraction of Annual Emissions by Month of Year", width=800)
fig.show()

Now lets see if there's a geographical relationship to these four distinct groupings. When mapped, it looks like ratio groupings are distinct for USA, US-East, Mexico, and Canada, so we can name them.

In [5]:
df_zones = df[(df["month"] == 1) & (df["year"] == 2000)].copy()
df_zones["Zone"] =  np.where(df_zones['ratio'] < 0.085, 'East',
                    np.where(df_zones['ratio'] < 0.095, 'USA',
                    np.where(df_zones['ratio'] < 0.098, 'Canada', 'Mexico')))

fig = px.scatter_geo(df_zones, lat="latitude", lon="longitude", color="Zone", projection="natural earth", fitbounds="locations", width=700, title="Unique Ratio Groups")
fig.update_traces(marker=dict(size=12))
fig.show()

Finally, let's extract a DataFrame with the zone for each location. This will be useful later.

In [6]:
zone_ids = df_zones[["loc_uid", "Zone"]]
zone_ids.head()

,loc_uid,Zone
372,"25.8, -97.8",Mexico
1392,"26.6, -81.7",USA
1548,"27.0, -80.5",USA
1332,"27.6, -82.3",USA
1296,"27.7, -82.4",USA


We also extract a DataFrame with the monthly ratio pattern for each zone.

In [7]:
df = df.merge(zone_ids, left_on=['loc_uid'], right_on=['loc_uid'])
ratios = df.groupby(["Zone", "month"]).mean().reset_index()[["Zone", "month", "ratio"]]
ratios.head()

,Zone,month,ratio
0,Canada,1,0.095629
1,Canada,2,0.088638
2,Canada,3,0.088606
3,Canada,4,0.075768
4,Canada,5,0.072535


## Yearly Model

Since we have a perfect way of allocating yearly data down to the monthly level, now our task becomes simpler. We just need a way to predict the yearly emmisions for each location in 2020. Let's take a look at the yearly data to see if there's any obvious trend.

In [8]:
df.sort_values(["loc_uid", "month"], inplace=True)
fig = px.line(df, x="year", y="co2_annual", color="loc_uid", title="Annual Emissions by Location, All History", width=800)
fig.show()

This looks pretty interesting! There's a straight line trend starting in 2012. Let's zoom in and take a closer look.

In [9]:
annual_totals = annual_totals[annual_totals["year"] >= 2012]
fig = px.line(annual_totals, x="year", y="co2", color="loc_uid", title="Annual Emissions by Location, Data Since 2012", width=800)
fig.show()

So to predict yearly emissions by location in 2020, we simply need to fit a linear model to these annual emissions data.

## Making Predictions

We now have everything we need to make predictions on the test dataset. First let's read in the test data and do some prep.

In [10]:
test_df = pd.read_csv("data/test.csv")
test_df['month'] = test_df['month'] + 1  # make month column start at 1 for January instead of 0
test_df['date']  = pd.to_datetime(test_df['year'].astype(str) + '-' + test_df['month'].astype(str) + '-01')
test_df['loc_uid'] = test_df['latitude'].astype(str) + ', ' + test_df['longitude'].astype(str)
test_df = test_df[["loc_uid", "latitude", "longitude", "date", "month", "year"]]
test_df.head()

,loc_uid,latitude,longitude,date,month,year
0,"33.3, -112.8",33.3,-112.8,2020-01-01,1,2020
1,"39.5, -112.5",39.5,-112.5,2020-01-01,1,2020
2,"36.9, -111.3",36.9,-111.3,2020-01-01,1,2020
3,"39.1, -111.1",39.1,-111.1,2020-01-01,1,2020
4,"39.3, -111.0",39.3,-111.0,2020-01-01,1,2020


We'll need the zones and ratios we identified earlier, so let's bring those in.

In [11]:
test_df = test_df.merge(zone_ids, left_on=['loc_uid'], right_on=['loc_uid'], how="left")
test_df = test_df.merge(ratios, left_on=['Zone', 'month'], right_on=['Zone', 'month'], how="left")
test_df.head()

,loc_uid,latitude,longitude,date,month,year,Zone,ratio
0,"33.3, -112.8",33.3,-112.8,2020-01-01,1,2020,USA,0.091867
1,"39.5, -112.5",39.5,-112.5,2020-01-01,1,2020,USA,0.091867
2,"36.9, -111.3",36.9,-111.3,2020-01-01,1,2020,USA,0.091867
3,"39.1, -111.1",39.1,-111.1,2020-01-01,1,2020,USA,0.091867
4,"39.3, -111.0",39.3,-111.0,2020-01-01,1,2020,USA,0.091867


Let's first make our prediction for annual emissions for each location in the training set. We'll use a linear model. 

In [12]:
def model(df, train):
    train = train[train["loc_uid"] == df["loc_uid"].iloc[0]]
    y = train[['co2']].values
    X = train[['year']].values
    lm = LinearRegression().fit(X, y)
    pred = lm.predict(df["year"].values.reshape(-1, 1))
    return pred[0][0]

loc_years_to_pred = test_df[["loc_uid", "year"]].drop_duplicates()
annual_preds = pd.DataFrame(loc_years_to_pred.groupby(["loc_uid", "year"]).apply(model, annual_totals)).reset_index()
annual_preds.columns = ["loc_uid", "year", "annual_pred"]
annual_preds.head()

,loc_uid,year,annual_pred
0,"25.8, -97.8",2020,56.498230
1,"26.6, -81.7",2020,55.149285
2,"27.0, -80.5",2020,133.156635
3,"27.6, -82.3",2020,67.552107
4,"27.9, -82.4",2020,63.407508


Now we need to convert these annual emissions to monthly emissions. To do this, we'll join the annual predictions back into our test DataFrame. Then we'll multiply by the ratios we identified earlier to convert annual to monthly emissions.

Finally we can produce our submission file.

In [13]:
test_df = test_df.merge(annual_preds, left_on=['loc_uid', 'year'], right_on=['loc_uid', 'year'], how="left")
test_df["monthly_pred"] = test_df["annual_pred"] * test_df["ratio"]

submission_df = test_df[["monthly_pred"]]
submission_df.columns = ["co2"]
submission_df.to_csv("submission.csv", index = False)
test_df.head()

,loc_uid,latitude,longitude,date,month,year,Zone,ratio,annual_pred,monthly_pred
0,"33.3, -112.8",33.3,-112.8,2020-01-01,1,2020,USA,0.091867,76.989204,7.072785
1,"39.5, -112.5",39.5,-112.5,2020-01-01,1,2020,USA,0.091867,126.466277,11.618107
2,"36.9, -111.3",36.9,-111.3,2020-01-01,1,2020,USA,0.091867,131.738887,12.102487
3,"39.1, -111.1",39.1,-111.1,2020-01-01,1,2020,USA,0.091867,84.915019,7.800908
4,"39.3, -111.0",39.3,-111.0,2020-01-01,1,2020,USA,0.091867,56.048926,5.149060


## Conclusions

This problem teaches us several important lessons. The first is about the importance of *data visualization*. By plotting this data, we were able to see clear trends and patterns in the emissions data. This deeper understanding of the data significantly influenced our approach. The second lesson is about the importance of *appropriate model selection*. In our case, a very simple linear regression model was sufficient to get us perfect results. While we experimented with more advanced algorithms (**TODO: SEE APPENDIX**), none of these was as effective as the simple approach.

We can conclude that the data used in this challenge (provided by the Copernicus Atmosphere Monitoring Service) is itself derived from a model. For more comprehensive efforts to model carbon emissions, it would be a good idea to investigate the methodology they use to generate this data. We should also investigate whether it's possible to break out CO2 emissions by source. Some categories of emissions (power plants, farms, and factories) are more amenable to point source carbon capture than diffuse sources (cars, planes, and household usage).

## Thanks

We would like to thank the SPE GCS Data Analytics Study Group for hosting this competition. We also specifically thank the organizers, Sri Poludasu and Neila Mazula, for their effort putting this together. Finally, we thank Shell for making the Xeek platform available for this competition.

## Appendix I - Prophet

In [14]:
from prophet import Prophet
import warnings
import logging
warnings.filterwarnings('ignore')
logging.getLogger('prophet').setLevel(logging.WARNING) 

train = df[df["year"] >= 2012].copy()
train.rename({"date": "ds", "co2_monthly": "y"}, axis='columns', inplace=True)

def predict_prophet(group, train):
    train = train[train["loc_uid"] == group["loc_uid"].iloc[0]]
    m = Prophet(seasonality_mode='multiplicative')
    m.add_regressor("ratio")
    m.fit(train)
    forecast = m.predict(group)    
    return forecast

test = test_df.copy()
test.rename({"date": "ds", "co2_monthly": "y"}, axis='columns', inplace=True)
result = test.groupby("loc_uid").apply(predict_prophet, train)
result = result[result["ds"] >= pd.to_datetime("2020-01-01")]
result.head()

c:\Users\lacha\Anaconda3\envs\spe\lib\site-packages\tqdm\auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



ds     trend  yhat_lower  yhat_upper  trend_lower  \
loc_uid                                                                   
25.8, -97.8 0 2020-01-01  4.650783    5.653081    5.657409     4.650697   
            1 2020-02-01  4.661895    5.416266    5.421740     4.660652   
            2 2020-03-01  4.672291    4.945929    4.952963     4.669831   
            3 2020-04-01  4.683404    4.707565    4.717205     4.679202   
            4 2020-05-01  4.694158    4.234457    4.246915     4.688287   

               trend_upper  extra_regressors_multiplicative  \
loc_uid                                                       
25.8, -97.8 0     4.650948                     2.496229e-01   
            1     4.663400                     1.872172e-01   
            2     4.675442                     6.240570e-02   
            3     4.688961                     3.233157e-09   
            4     4.701618                    -1.248114e-01   

               extra_regressors_multiplicative_lower  \
loc_uid                                                
25.8, -97.8 0                           2.496229e-01   
            1                           1.872172e-01   
            2                           6.240570e-02   
            3                           3.233157e-09   
            4                          -1.248114e-01   

               extra_regressors_multiplicative_upper  multiplicative_terms  \
loc_uid                                                                      
25.8, -97.8 0                           2.496229e-01              0.215990   
            1                           1.872172e-01              0.162371   
            2                           6.240570e-02              0.059249   
            3                           3.233157e-09              0.006099   
            4                          -1.248114e-01             -0.096760   

               ...         ratio   ratio_lower   ratio_upper    yearly  \
loc_uid        ...                                                       
25.8, -97.8 0  ...  2.496229e-01  2.496229e-01  2.496229e-01 -0.033633   
            1  ...  1.872172e-01  1.872172e-01  1.872172e-01 -0.024847   
            2  ...  6.240570e-02  6.240570e-02  6.240570e-02 -0.003157   
            3  ...  3.233157e-09  3.233157e-09  3.233157e-09  0.006099   
            4  ... -1.248114e-01 -1.248114e-01 -1.248114e-01  0.028051   

               yearly_lower  yearly_upper  additive_terms  \
loc_uid                                                     
25.8, -97.8 0     -0.033633     -0.033633             0.0   
            1     -0.024847     -0.024847             0.0   
            2     -0.003157     -0.003157             0.0   
            3      0.006099      0.006099             0.0   
            4      0.028051      0.028051             0.0   

               additive_terms_lower  additive_terms_upper      yhat  
loc_uid                                                              
25.8, -97.8 0                   0.0                   0.0  5.655307  
            1                   0.0                   0.0  5.418850  
            2                   0.0                   0.0  4.949120  
            3                   0.0                   0.0  4.711967  
            4                   0.0                   0.0  4.239950  

[5 rows x 22 columns]

In [15]:
pred_df = result[["ds", "yhat"]].reset_index().drop("level_1", axis="columns")
pred_df.head()
test_df_new = test_df.merge(pred_df, left_on=['loc_uid', 'date'], right_on=['loc_uid', 'ds'], how="left")
test_df_new.head()

,loc_uid,latitude,longitude,date,month,year,Zone,ratio,annual_pred,monthly_pred,ds,yhat
0,"33.3, -112.8",33.3,-112.8,2020-01-01,1,2020,USA,0.091867,76.989204,7.072785,2020-01-01,7.075078
1,"39.5, -112.5",39.5,-112.5,2020-01-01,1,2020,USA,0.091867,126.466277,11.618107,2020-01-01,11.618652
2,"36.9, -111.3",36.9,-111.3,2020-01-01,1,2020,USA,0.091867,131.738887,12.102487,2020-01-01,12.106419
3,"39.1, -111.1",39.1,-111.1,2020-01-01,1,2020,USA,0.091867,84.915019,7.800908,2020-01-01,7.803441
4,"39.3, -111.0",39.3,-111.0,2020-01-01,1,2020,USA,0.091867,56.048926,5.149060,2020-01-01,5.150729


In [16]:
print("RMSE: " + str(((test_df_new["monthly_pred"] - test_df_new["yhat"]) ** 2).mean() ** .5))

RMSE: 0.0018723000988688494


## Appendix II - XGBoost

In [24]:
import xgboost as xgb

train = df[df["year"] >= 2012].copy()
train.rename({"date": "ds", "co2_monthly": "y"}, axis='columns', inplace=True)

def predict_xgb(group, train):
    train = train[train["loc_uid"] == group["loc_uid"].iloc[0]]
    train = train[["ds", "y"]]
    param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic'}
    print(train.head())
    #bst = xgb.train(param, train.values, 2)
    #group = group[["ds"]]
    #preds = bst.predict(group.values)
    #return preds

test = test_df.copy()
test.rename({"date": "ds", "co2_monthly": "y"}, axis='columns', inplace=True)
result = test.groupby("loc_uid").apply(predict_xgb, train)
#result.head()

           ds         y
12 2012-01-01  4.402636
13 2013-01-01  4.558534
14 2014-01-01  4.714432
15 2015-01-01  4.870331
16 2016-01-01  5.026229
            ds         y
252 2012-01-01  5.678810
253 2013-01-01  5.602260
254 2014-01-01  5.525710
255 2015-01-01  5.449161
256 2016-01-01  5.372611
            ds          y
492 2012-01-01  11.874272
493 2013-01-01  11.919079
494 2014-01-01  11.963887
495 2015-01-01  12.008694
496 2016-01-01  12.053501
            ds         y
732 2012-01-01  6.986821
733 2013-01-01  6.889197
734 2014-01-01  6.791572
735 2015-01-01  6.693948
736 2016-01-01  6.596323
             ds         y
1116 2012-01-01  6.558151
1117 2013-01-01  6.466516
1118 2014-01-01  6.374881
1119 2015-01-01  6.283246
1120 2016-01-01  6.191611
             ds         y
1356 2012-01-01  8.416478
1357 2013-01-01  8.727786
1358 2014-01-01  9.039094
1359 2015-01-01  9.350402
1360 2016-01-01  9.661710
             ds         y
1596 2012-01-01  5.929094
1597 2013-01-01  5.848044
1598 2014-

,loc_uid,latitude,longitude,ds,month,year,y,co2_annual,ratio,Zone
12,"25.8, -97.8",25.8,-97.8,2012-01-01,1,2012,4.402636,44.026359,0.1,Mexico
13,"25.8, -97.8",25.8,-97.8,2013-01-01,1,2013,4.558534,45.585343,0.1,Mexico
14,"25.8, -97.8",25.8,-97.8,2014-01-01,1,2014,4.714432,47.144327,0.1,Mexico
15,"25.8, -97.8",25.8,-97.8,2015-01-01,1,2015,4.870331,48.703311,0.1,Mexico
16,"25.8, -97.8",25.8,-97.8,2016-01-01,1,2016,5.026229,50.262294,0.1,Mexico
